In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "train_25_mb/"
create_directory(data_directory)
create_directory("models/train_25_mb/")
create_directory("scalers")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [4]:
BATCH_SIZE = 128
EPOCHS = 60
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 2700

INPUT_DIM = len(BASIC_TRAINING_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5")
val_trials = read_dataset("data/val_passive_trials.h5")

100%|██████████| 1000/1000 [00:08<00:00, 112.97it/s]


## Train model and plot loss and accuracy

In [6]:
N_MODELS = 25

stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device)


for seed in range(N_MODELS):
    df = pd.DataFrame()

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    torch.save(best_mass_model.state_dict(), "models/train_25_mb/best_mass_model_seed_%d.pt" % seed)
    torch.save(best_force_model.state_dict(), "models/train_25_mb/best_force_model_seed_%d.pt" % seed)
    
    
    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1]) 
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 1000/1000 [00:01<00:00, 724.57it/s]
Train_loss: ([0.45567036 0.58159053]) Train_acc: ([80.68571429 72.94285714]) Val_acc: ([65.6 65.9]): 100%|██████████| 60/60 [12:33<00:00, 12.42s/it]
Train_loss: ([0.90741879 0.94689551]) Train_acc: ([52.57142857 46.08571429]) Val_acc: ([48.8 47.3]): 100%|██████████| 60/60 [12:25<00:00, 12.41s/it]
Train_loss: ([0.61590621 0.57603539]) Train_acc: ([72.77142857 70.77142857]) Val_acc: ([65.  59.1]): 100%|██████████| 60/60 [12:24<00:00, 12.44s/it]
Train_loss: ([0.65550695 0.70963457]) Train_acc: ([69.11428571 62.28571429]) Val_acc: ([61.6 59.5]): 100%|██████████| 60/60 [12:26<00:00, 12.44s/it]
Train_loss: ([0.44278081 0.51179188]) Train_acc: ([74.91428571 71.54285714]) Val_acc: ([60.6 56.9]): 100%|██████████| 60/60 [12:25<00:00, 12.41s/it]
Train_loss: ([0.4865056  0.49947382]) Train_acc: ([79.31428571 71.54285714]) Val_acc: ([64.5 63. ]): 100%|██████████| 60/60 [12:23<00:00, 12.40s/it]
Train_loss: ([0.3280748  0.37600666]) Train_acc: ([88

## Save model and scaler

In [7]:
joblib.dump(scaler, "scalers/passive_dual_scaler.sk")

['scalers/passive_dual_scaler.sk']